# Load dependencies

In [2]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn import preprocessing #for one-hot encoding

For the Live SQL Server Authentication, please enter your


# SQL Script

In [10]:
s='''
    
        SELECT DISTINCT*

        FROM(
        SELECT 
        [StudyID]
        ,[ICDChapter]
        from [CPMS_BI].[dbo].[ODP_StudyICD]) AS IC

        LEFT JOIN 
        (SELECT 
        StudyID
        ,[StudyStatus]
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria


        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        AND Study_IsCommercial = 0
        --AND Study_IsDraft=0
        ) AS ST
        ON IC.StudyID = ST.StudyID

        WHERE 1=1
        AND ST.StudyID is NOT NULL
        ORDER BY ST.StudyID
    '''

# with open("C:/Users/busaji/Documents/Projects/StudyClassifier-main/dat/sql-read.txt","r") as f:
#     s = f.read()

In [11]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_31664\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [12]:
study_df.dtypes

ICDBlock                  object
StudyShortName            object
StudyTitle                object
StudyResearchSummary      object
StudyInclusionCriteria    object
dtype: object

In [13]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [14]:
study_df2

,ICDBlock,StudyShortName,StudyTitle,StudyResearchSummary,StudyInclusionCriteria
0,not available,MCRN046 (MPS VI CSP),MPS VI Clinical Surveillance Program,not available,Please see study protocol
1,not available,MCRN062 (NAI113678),"An open-label, multi-centre, single arm study ...",not available,Please see study protocol
2,not available,MCRN080 (REMICADEPIB4003),"A Multicenter, Prospective Long-term Registry ...",not available,Please see study protocol
3,not available,NCRN163 COMPLEMENT - Ofatumumab + Bendamustine...,"A Randomized, Open Label Study of Ofatumumab a...",not available,1. Indolent lymphoma including grades 1-3a fol...
4,not available,NCRN177 - ACE: At home Compression Evaluation,At Home Evaluation of Two Pneumatic Compressio...,not available,•Subjects must be ≥ 18 years old or legal age ...
...,...,...,...,...,...
6837,not available,A Study of SGN-MesoC2 in Advanced Solid Tumors,"A Phase 1 Open-label, Multicenter Study to Eva...",not available,Subjects are eligible to be included in the st...
6838,not available,EIK1001-006,"A Multicenter, Randomized, Double-Blind, Activ...",not available,"""To be eligible for inclusion in this study, p..."
6839,not available,MK2870-027,A Phase 1/2 Open-label Clinical Study to Evalu...,not available,Type of Participant and Disease Characteristic...
6840,not available,INO-TRANSIT,Real-world treatment patterns and clinical out...,Acute lymphoblastic leukemia (ALL) is the fast...,Patients must meet all of the following inclus...


In [51]:
#workaround - but 'reload' is not defined
import sys
reload(sys)
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

1. Convert to string

In [60]:
#study_df2['StudyResearchSummary'] = study_df2['StudyResearchSummary'].encode('utf-8')

for index, row in study_df2.iterrows():
    row['StudyResearchSummary'] = row['StudyResearchSummary'].encode('utf-8')




# TF-IDF Vectorizer

In [61]:

corpus = study_df2['StudyResearchSummary']
vectorizer = TfidfVectorizer(lowercase = True, encoding = "utf-8", decode_error="ignore", stop_words='english', analyzer='word', ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X.shape)

(19513, 956142)


In [62]:
X.head()

AttributeError: 'csr_matrix' object has no attribute 'head'

# K-Means Clustering

In [63]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(X)


In [64]:
feat_nms = kmeans.get_feature_names_out()

In [65]:
feat_nms

array(['kmeans0', 'kmeans1', 'kmeans2', 'kmeans3', 'kmeans4', 'kmeans5',
       'kmeans6', 'kmeans7', 'kmeans8', 'kmeans9', 'kmeans10', 'kmeans11',
       'kmeans12', 'kmeans13', 'kmeans14', 'kmeans15', 'kmeans16',
       'kmeans17', 'kmeans18', 'kmeans19'], dtype=object)

In [67]:
kmeans.

KMeans(n_clusters=20, random_state=0)

# LDA

In [66]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

lda = LatentDirichletAllocation(n_components=5,
    random_state=0)
lda.fit(X) #fit X


LatentDirichletAllocation(n_components=5, random_state=0)

In [68]:
#transform X
X_lda =lda.transform(X)



In [71]:
X_lda.shape

(19513, 5)

In [ ]:
#get output as pandas
X_lda.set_output(*,transform="pandas")